In [1]:
from torch import nn

class SRCNN(nn.Module):  #搭建SRCNN 3层卷积模型，Conve2d（输入层数，输出层数，卷积核大小，步长，填充层）
    def __init__(self, num_channels=1):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=9 // 2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=5 // 2)
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding=5 // 2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

In [2]:
import torch
import numpy as np

"""
       只操作y通道
       因为我们感兴趣的不是颜色变化(存储在 CbCr 通道中的信息)而只是其亮度(Y 通道);
       根本原因在于相较于色差，人类视觉对亮度变化更为敏感。
"""
def convert_rgb_to_y(img):
    if type(img) == np.ndarray:
        return 16. + (64.738 * img[:, :, 0] + 129.057 * img[:, :, 1] + 25.064 * img[:, :, 2]) / 256.
    elif type(img) == torch.Tensor:
        if len(img.shape) == 4:
            img = img.squeeze(0)
        return 16. + (64.738 * img[0, :, :] + 129.057 * img[1, :, :] + 25.064 * img[2, :, :]) / 256.
    else:
        raise Exception('Unknown Type', type(img))

"""
        RGB转YCBCR
        Y=0.257*R+0.564*G+0.098*B+16
        Cb=-0.148*R-0.291*G+0.439*B+128
        Cr=0.439*R-0.368*G-0.071*B+128
"""
def convert_rgb_to_ycbcr(img):
    if type(img) == np.ndarray:
        y = 16. + (64.738 * img[:, :, 0] + 129.057 * img[:, :, 1] + 25.064 * img[:, :, 2]) / 256.
        cb = 128. + (-37.945 * img[:, :, 0] - 74.494 * img[:, :, 1] + 112.439 * img[:, :, 2]) / 256.
        cr = 128. + (112.439 * img[:, :, 0] - 94.154 * img[:, :, 1] - 18.285 * img[:, :, 2]) / 256.
        return np.array([y, cb, cr]).transpose([1, 2, 0])
    elif type(img) == torch.Tensor:
        if len(img.shape) == 4:
            img = img.squeeze(0)
        y = 16. + (64.738 * img[0, :, :] + 129.057 * img[1, :, :] + 25.064 * img[2, :, :]) / 256.
        cb = 128. + (-37.945 * img[0, :, :] - 74.494 * img[1, :, :] + 112.439 * img[2, :, :]) / 256.
        cr = 128. + (112.439 * img[0, :, :] - 94.154 * img[1, :, :] - 18.285 * img[2, :, :]) / 256.
        return torch.cat([y, cb, cr], 0).permute(1, 2, 0)
    else:
        raise Exception('Unknown Type', type(img))

"""
        YCBCR转RGB
        R=1.164*(Y-16)+1.596*(Cr-128)
        G=1.164*(Y-16)-0.392*(Cb-128)-0.813*(Cr-128)
        B=1.164*(Y-16)+2.017*(Cb-128)
"""
def convert_ycbcr_to_rgb(img):
    if type(img) == np.ndarray:
        r = 298.082 * img[:, :, 0] / 256. + 408.583 * img[:, :, 2] / 256. - 222.921
        g = 298.082 * img[:, :, 0] / 256. - 100.291 * img[:, :, 1] / 256. - 208.120 * img[:, :, 2] / 256. + 135.576
        b = 298.082 * img[:, :, 0] / 256. + 516.412 * img[:, :, 1] / 256. - 276.836
        return np.array([r, g, b]).transpose([1, 2, 0])
    elif type(img) == torch.Tensor:
        if len(img.shape) == 4:
            img = img.squeeze(0)
        r = 298.082 * img[0, :, :] / 256. + 408.583 * img[2, :, :] / 256. - 222.921
        g = 298.082 * img[0, :, :] / 256. - 100.291 * img[1, :, :] / 256. - 208.120 * img[2, :, :] / 256. + 135.576
        b = 298.082 * img[0, :, :] / 256. + 516.412 * img[1, :, :] / 256. - 276.836
        return torch.cat([r, g, b], 0).permute(1, 2, 0)
    else:
        raise Exception('Unknown Type', type(img))

# PSNR 计算
def calc_psnr(img1, img2):
    return 10. * torch.log10(1. / torch.mean((img1 - img2) ** 2))

# 计算 平均数，求和，长度
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [4]:
import argparse

import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image

if __name__ == '__main__':
    # 设置权重参数目录，处理图像目录，放大倍数
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights-file', default='pth/x3/best.pth', type=str)
    parser.add_argument('--image-file', default='Set5/woman.png', type=str)
    parser.add_argument('--scale', type=int, default=3)
    args = parser.parse_args()
    #  Benchmark模式会提升计算速度
    cudnn.benchmark = True
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model = SRCNN().to(device)   # 新建一个模型

    state_dict = model.state_dict()  # 通过 model.state_dict()得到模型有哪些 parameters and persistent buffers
    # torch.load('tensors.pth', map_location=lambda storage, loc: storage)  使用函数将所有张量加载到CPU（适用在GPU训练的模型在CPU上加载）
    for n, p in torch.load(args.weights_file, map_location=lambda storage, loc: storage).items():   # 载入最好的模型参数
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

    model.eval()   # 切换为测试模式 ，取消dropout

    image = pil_image.open(args.image_file).convert('RGB')   # 将图片转为RGB类型

    # 经过一个插值操作，首先将原始图片重设尺寸，使之可以被放大倍数scale整除
    # 得到低分辨率图像Lr，即三次插值后的图像，同时保存输出
    image_width = (image.width // args.scale) * args.scale
    image_height = (image.height // args.scale) * args.scale
    image = image.resize((image_width, image_height), resample=pil_image.Resampling.BICUBIC)
    image = image.resize((image.width // args.scale, image.height // args.scale), resample=pil_image.Resampling.BICUBIC)
    image = image.resize((image.width * args.scale, image.height * args.scale), resample=pil_image.Resampling.BICUBIC)
    image.save(args.image_file.replace('.', '_bicubic{}.'.format(args.scale)))
    # 将图像转化为数组类型，同时图像转为ycbcr类型
    image = np.array(image).astype(np.float32)
    ycbcr = convert_rgb_to_ycbcr(image)
    # 得到 ycbcr中的 y 通道
    y = ycbcr[..., 0]
    y /= 255.  # 归一化处理
    y = torch.from_numpy(y).to(device) #把数组转换成张量，且二者共享内存，对张量进行修改比如重新赋值，那么原始数组也会相应发生改变，并且将参数放到device上
    y = y.unsqueeze(0).unsqueeze(0)  # 增加两个维度
    # 令reqires_grad自动设为False，关闭自动求导
    # clamp将inputs归一化为0到1区间
    with torch.no_grad():
        preds = model(y).clamp(0.0, 1.0)

    psnr = calc_psnr(y, preds)   # 计算y通道的psnr值
    print('PSNR: {:.2f}'.format(psnr))  # 格式化输出PSNR值

    # 1.mul函数类似矩阵.*，即每个元素×255
    # 2. *.cpu（）.numpy（） 将数据的处理设备从其他设备（如gpu拿到cpu上），不会改变变量类型，转换后仍然是Tensor变量，同时将Tensor转化为ndarray
    # 3. *.squeeze(0).squeeze(0)数据的维度进行压缩
    preds = preds.mul(255.0).cpu().numpy().squeeze(0).squeeze(0)  #得到的是经过模型处理，取值在[0,255]的y通道图像

    # 将img的数据格式由（channels,imagesize,imagesize）转化为（imagesize,imagesize,channels）,进行格式的转换后方可进行显示。
    output = np.array([preds, ycbcr[..., 1], ycbcr[..., 2]]).transpose([1, 2, 0])

    output = np.clip(convert_ycbcr_to_rgb(output), 0.0, 255.0).astype(np.uint8)  # 将图像格式从ycbcr转为rgb，限制取值范围[0,255]，同时矩阵元素类型为uint8类型
    output = pil_image.fromarray(output)   # array转换成image，即将矩阵转为图像
    output.save(args.image_file.replace('.', '_srcnn{}.'.format(args.scale)))  # 对图像进行保存


usage: ipykernel_launcher.py [-h] [--weights-file WEIGHTS_FILE] [--image-file IMAGE_FILE] [--scale SCALE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\ASUS\AppData\Roaming\jupyter\runtime\kernel-227eb4f0-9cc7-4c94-90b7-72b77303409c.json


SystemExit: 2

F:\Anaconda\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 计算PSNR时，常常将原始的高分辨率图像和SRCNN生成的高分辨率图像进行对比。
# 这是因为SRCNN的目标是将低分辨率图像升采样到与原始高分辨率图像尽可能接近的分辨率，PSNR可以帮助量化生成图像与原始高分辨率图像之间的距离。
# 加载原始高分辨率图像和生成图像
original_img = Image.open('Set5/baby.png').convert('RGB')
generated_img = Image.open('Set5/baby_srcnn3.png').convert('RGB')
generated_img = generated_img.resize((512, 512), resample=Image.Resampling.BICUBIC)
# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_baby:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_baby:', ssim_score)

PSNR_baby: 33.48119486829164
SSIM_baby: 0.87122613


In [6]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 计算PSNR时，常常将原始的高分辨率图像和SRCNN生成的高分辨率图像进行对比。
# 这是因为SRCNN的目标是将低分辨率图像升采样到与原始高分辨率图像尽可能接近的分辨率，PSNR可以帮助量化生成图像与原始高分辨率图像之间的距离。
# 加载原始高分辨率图像和生成图像
original_img = Image.open('Set5/bird.png').convert('RGB')
generated_img = Image.open('Set5/bird_srcnn3.png').convert('RGB')
generated_img = generated_img.resize((288, 288), resample=Image.Resampling.BICUBIC)
# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_bird:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_bird:', ssim_score)

PSNR_bird: 32.53850629430682
SSIM_bird: 0.88938755


In [7]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 计算PSNR时，常常将原始的高分辨率图像和SRCNN生成的高分辨率图像进行对比。
# 这是因为SRCNN的目标是将低分辨率图像升采样到与原始高分辨率图像尽可能接近的分辨率，PSNR可以帮助量化生成图像与原始高分辨率图像之间的距离。
# 加载原始高分辨率图像和生成图像
original_img = Image.open('Set5/butterfly.png').convert('RGB')
generated_img = Image.open('Set5/butterfly_srcnn3.png').convert('RGB')
generated_img = generated_img.resize((256, 256), resample=Image.Resampling.BICUBIC)
# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_butterfly:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_butterfly:', ssim_score)

PSNR_butterfly: 26.22851089195516
SSIM_butterfly: 0.7930804


In [8]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 计算PSNR时，常常将原始的高分辨率图像和SRCNN生成的高分辨率图像进行对比。
# 这是因为SRCNN的目标是将低分辨率图像升采样到与原始高分辨率图像尽可能接近的分辨率，PSNR可以帮助量化生成图像与原始高分辨率图像之间的距离。
# 加载原始高分辨率图像和生成图像
original_img = Image.open('Set5/head.png').convert('RGB')
generated_img = Image.open('Set5/head_srcnn3.png').convert('RGB')
generated_img = generated_img.resize((280, 280), resample=Image.Resampling.BICUBIC)
# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_head:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_head:', ssim_score)

PSNR_head: 30.44916296455154
SSIM_head: 0.6903467


In [9]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 计算PSNR时，常常将原始的高分辨率图像和SRCNN生成的高分辨率图像进行对比。
# 这是因为SRCNN的目标是将低分辨率图像升采样到与原始高分辨率图像尽可能接近的分辨率，PSNR可以帮助量化生成图像与原始高分辨率图像之间的距离。
# 加载原始高分辨率图像和生成图像
original_img = Image.open('Set5/woman.png').convert('RGB')
generated_img = Image.open('Set5/woman_srcnn3.png').convert('RGB')
generated_img = generated_img.resize((228, 344), resample=Image.Resampling.BICUBIC)
# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_woman:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_woman:', ssim_score)

PSNR_woman: 29.721867585347375
SSIM_woman: 0.8667073
